**Zachary Torstrick**  
**HW 5**  
**ATMS 523**
**Fall 2025**  

## Module 5 Project

Fork this repository, and submit this code as a pull request back to GitHub by the date and time listed in Canvas.

For this assignment, use the dataset called `radar_parameters.csv` provided in the GitHub repository in the folder `homework`.

### Dataset Description

The training data consists of polarimetric radar parameters calculated from a disdrometer (an instrument that measures rain drop sizes, shapes, and rainfall rate) measurements from several years in Huntsville, Alabama. A model called `pytmatrix` is used to calculate polarimetric radar parameters from the droplet observations, which can be used as a way to compare what a remote sensing instrument would see and rainfall.

### Data columns

Features (radar measurements):

`Zh` - radar reflectivity factor (dBZ) - use the formula $dBZ = 10\log_{10}(Z)$

`Zdr` - differential reflectivity

`Ldr` - linear depolarization ratio

`Kdp` - specific differential phase

`Ah` - specific attenuation

`Adp` - differential attenuation

Target :

`R` - rain rate

In [15]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

df = pd.read_csv('homework/radar_parameters.csv')

X = df.drop('R (mm/hr)', axis=1)
y = df['R (mm/hr)']   

##############################################################################
## 1. Split the data into a 70-30 split for training and testing data.
##############################################################################
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    random_state=0,
    train_size=0.7)

##############################################################################
### 2. Using the split created in (1), train a multiple linear regression dataset 
###    using the training dataset, and validate it using the testing dataset.  
###    Compare the $R^2$ and root mean square errors of model on the training and 
###    testing sets to a baseline prediction of rain rate using the formula 
###    $Z = 200 R^{1.6}$.
##############################################################################

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predict_test = model.predict(X_test)
predict_train = model.predict(X_train)

# Evaluate R² and RMSE
r2_train = r2_score(y_train, predict_train)
rmse_train = np.sqrt(mean_squared_error(y_train, predict_train))
r2_test = r2_score(y_test, predict_test)
rmse_test = np.sqrt(mean_squared_error(y_test, predict_test))

# Convert dBZ to linear Z
# dBZ = 10 * log10(Z)  →  Z = 10^(dBZ/10)
Zh_train_dBZ = X_train['Zh (dBZ)']
Zh_test_dBZ = X_test['Zh (dBZ)']
Z_train = 10 ** (Zh_train_dBZ / 10)
Z_test = 10 ** (Zh_test_dBZ / 10)

# Solve for R: R = (Z / 200)^(1/1.6)
baseline_train = (Z_train / 200) ** (1 / 1.6)
baseline_test = (Z_test / 200) ** (1 / 1.6)

# predict baseline
r2_baseline_train = r2_score(y_train, baseline_train)
rmse_baseline_train = np.sqrt(mean_squared_error(y_train, baseline_train))
r2_baseline_test = r2_score(y_test, baseline_test)
rmse_baseline_test = np.sqrt(mean_squared_error(y_test, baseline_test))

print("=== Baseline (Z-R Relationship) ===")
print(f"Training R²: {r2_baseline_train:.4f}, RMSE: {rmse_baseline_train:.4f}")
print(f"Testing R²: {r2_baseline_test:.4f}, RMSE: {rmse_baseline_test:.4f}")
print('\n')
print("=== Linear Regression ===")
print(f"Training R²: {r2_train:.4f}, RMSE: {rmse_train:.4f}")
print(f"Testing R²: {r2_test:.4f}, RMSE: {rmse_test:.4f}")


=== Baseline (Z-R Relationship) ===
Training R²: 0.3325, RMSE: 7.0725
Testing R²: 0.2266, RMSE: 7.3524


=== Linear Regression ===
Training R²: 0.9888, RMSE: 0.9147
Testing R²: 0.9869, RMSE: 0.9583


In [ ]:
##############################################################################
### 3. Repeat 1 doing a grid search over polynomial orders, using a grid 
###    search over orders 0-9, and use cross-validation of 7 folds.  For the 
###    best polynomial model in terms of $R^2$, does it outperform the baseline 
###    and the linear regression model in terms of $R^2$ and root mean square error?
##############################################################################












4. Repeat 1 with a Random Forest Regressor, and perform a grid_search on the following parameters:
   
   ```python
   param_grid = {
    "bootstrap": [True, False],
    "max_depth": [10, 100],
    "max_features": ["sqrt", 1.0],  
    "min_samples_leaf": [1, 4],
    "min_samples_split": [2, 10],
    "n_estimators": [200, 1000]}
   ```
  Can you beat the baseline, or the linear regression, or best polynomial model with the best optimized Random Forest Regressor in terms of $R^2$ and root mean square error?